In [1]:
!pip install -U dill==0.3.3 pandas==1.1.5 numpy==1.19.5  scikit-learn==0.22.2 nltk==3.2.5 torch==1.7.1 torchtext==0.3.1
# !pip install -U torchtext==0.10.0
# !pip install -U torch==1.8.0 torchtext==0.9.0
# !pip install -U torch==1.7.1 torchtext==0.3.1
# !pip install torchtext==0.9.0 --quiet
# !pip install torchvision==0.9.0 --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.0 MB/s 
     |████████████████████████████████| 9.5 MB 30.8 MB/s 
     |████████████████████████████████| 14.8 MB 32.9 MB/s 
     |████████████████████████████████| 7.1 MB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.1 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 62 kB 975 kB/s 
  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392156 sha256=5857c5b4127fa5f13e953eff00a0e50a27dc1cd648d239882316e4d589fa3771
  Stored in directory: /root/.cache/pip/wheels/60/de/57/6bced01d340818a36413222e6efcc7766d1f1e4575782b6223
Successfully built nltk
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: torch
    Found existing installation: torch 

In [2]:
# dill.__version__

In [63]:
import dill
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
import pandas as pd

import torch
import torch.nn as nn
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
import torch
# print(torch.__version__)
# print(torchtext.__version__)
# from torch.utils.tensorboard import SummaryWriter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [60]:
%cd /gdrive/MyDrive/Deep-Project/
!cp data_processed.zip /content
%cd /content
!mkdir -p /content/processed/
!unzip -q data_processed.zip
# !curl -L -O https://github.com/bjpublic/DeepLearningProject/raw/main/08_%EC%88%98%EB%8A%A5_%EC%98%81%EC%96%B4_%ED%92%80%EA%B8%B0/data/raw/sat_problems.csv


/gdrive/MyDrive/Deep-Project
/content


In [6]:
DATA_PATH = '/content/data/processed/'
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [7]:
class LSTMClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()
        self.embed_layer = nn.Embedding(num_embeddings = num_embeddings, embedding_dim = embedding_dim,
                                        padding_idx = pad_idx)
        self.lstm_layer = nn.LSTM(input_size = embedding_dim, hidden_size= hidden_size,
                                  bidirectional= True, dropout=.5)
        self.last_layer = nn.Sequential(
            nn.Linear(hidden_size *2, hidden_size),
            nn.Dropout(.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size,1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        embed_x = self.embed_layer(x)
        output, (_,_) = self.lstm_layer(embed_x)
        last_output = output[:, -1, :]              #LSTM output = (batch_size, embed_size, output_size), 가장 마지막 단어만 사용
        last_output = self.last_layer(last_output)
        return last_output

In [56]:
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()
        self.embed_layer = nn.Embedding(num_embeddings = num_embeddings, embedding_dim = embedding_dim,
                                        padding_idx = pad_idx)
        self.lstm_layer = nn.LSTM(input_size = embedding_dim, hidden_size= hidden_size,
                                  bidirectional= True, dropout=.5, batch_first = True)
        self.last_layer = nn.Sequential(
            nn.Linear(hidden_size *2, 1),
            nn.Dropout(.5),
            nn.Sigmoid(),
        )
    def forward(self, x):
        embed_x = self.embed_layer(x)
        output, _ = self.lstm_layer(embed_x)

        pool = nn.functional.max_pool1d(output.transpose(1,2), embed_x.shape[1])
        pool = pool.transpose(1,2).squeeze()
        output = self.last_layer(pool)
        return output.squeeze()
# class LSTMPoolingClassifier(nn.Module):
#     def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
#         super(LSTMPoolingClassifier, self).__init__()
#         self.embed_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=pad_idx)
#         self.hidden_size = hidden_size
#         self.embedding_dim = embedding_dim
#         self.num_layers = num_layers
#         self.ih2h = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
#                             bidirectional=True, batch_first=True, dropout=0.5)
#         self.pool2o = nn.Linear(2 * hidden_size, 1)
#         self.sigmoid = nn.Sigmoid()
#         self.softmax = nn.Softmax()
#         self.dropout = nn.Dropout(p=0.5)

#     def forward(self, x):
#         x = self.embed_layer(x)
#         o, _ = self.ih2h(x)
#         pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1])
#         pool = pool.transpose(1, 2).squeeze()
#         pool = self.dropout(pool)
#         output = self.sigmoid(self.pool2o(pool))
#         return output.squeeze()

In [10]:
TEXT = Field(
    sequential=True,
    use_vocab = True,
    tokenize= word_tokenize,
    lower = True,
    batch_first = True
)

LABEL=Field(
    sequential = False,
    use_vocab = False,
    batch_first = True,
)
sat_train_data, sat_valid_data, sat_test_data = TabularDataset.splits(path='/content/data/processed/',train='sat_train.tsv',
                                                                      validation = 'sat_valid.tsv', test = 'sat_test.tsv',
                                                                      format = 'tsv', fields =[('text', TEXT), ('label',LABEL)],
                                                                      skip_header=1,)
sat_train_iterator, sat_valid_iterator, sat_test_iterator = BucketIterator.splits((sat_train_data, sat_valid_data, sat_test_data),
                                                                                  batch_size = 8,
                                                                                  device = None,
                                                                                  sort = False,)
TEXT.build_vocab(sat_train_data, min_freq =2)

In [11]:
cola_train_data, cola_valid_data, cola_test_data = TabularDataset.splits(path='/content/data/processed/',train='cola_train.tsv',
                                                                      validation = 'cola_valid.tsv', test = 'cola_test.tsv',
                                                                      format = 'tsv', fields =[('text', TEXT), ('label',LABEL)],
                                                                      skip_header=1,)
cola_train_iterator, cola_valid_iterator, cola_test_iterator = BucketIterator.splits((cola_train_data, cola_valid_data, cola_test_data),
                                                                                  batch_size = 32,
                                                                                  device = None,
                                                                                  sort = False,)
TEXT.build_vocab(cola_train_data, min_freq =2)

In [12]:
# sat_train_iterator
# next(iter(cola_train_iterator)).text
# # print(batch.text)

In [78]:
def train(model, train_loader, optimizer, criterion, device):

    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        text = batch.text
        if text.shape[0] >1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            writer.add_scalar('training loss',
                loss,
                epoch * len(train_loader) + i)
            
    return epoch_loss / len(train_loader)

def evaluate(model, valid_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(valid_loader):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            epoch_loss += loss.item()

            writer.add_scalar('training loss',
                loss,
                epoch * len(valid_loader) + i)

    return epoch_loss / len(valid_loader)

In [79]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_classifier = LSTMClassifier(
    num_embeddings = len(TEXT.vocab),
    embedding_dim = 100,
    hidden_size = 200,
    num_layers = 4,
    pad_idx = PAD_IDX
)

lstm_pool_classifier = LSTMPoolingClassifier(
    num_embeddings = len(TEXT.vocab),
    embedding_dim = 100,
    hidden_size = 400,
    num_layers = 4,
    pad_idx = PAD_IDX
)

device = 'cpu'
if torch.cuda.is_available() :
    device = "cuda"

_ = lstm_classifier.to(device)
optimizer = torch.optim.Adam(lstm_classifier.parameters())
bce_loss_fn = nn.BCELoss()

_ = lstm_pool_classifier.to(device)
optimizer_pool = torch.optim.Adam(lstm_pool_classifier.parameters(), lr=0.01)
bce_loss_fn_pool = nn.BCELoss()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [80]:
mode = 'sat_'
dataset = cola_train_iterator
valid_set = cola_valid_iterator
# train_model = lstm_classifier


if mode ==  'sat_':
    dataset = sat_train_iterator
    valid_set = sat_valid_iterator
elif mode == 'pool':
    train_model = lstm_pool_classifier
    optimizer = optimizer_pool
    bce_loss_fn = bce_loss_fn_pool

for epoch in range(20):
    train_loss = train(train_model, dataset, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(train_model, valid_set, bce_loss_fn, device)
    print(f'Epoch : {epoch +1 :02}    Train loss : {train_loss : .5f}   val loss : {valid_loss : .5f}')


Epoch : 01    Train loss :  0.40800   val loss :  0.34309
Epoch : 02    Train loss :  0.37597   val loss :  0.34309
Epoch : 03    Train loss :  0.37399   val loss :  0.34309
Epoch : 04    Train loss :  0.39258   val loss :  0.34309
Epoch : 05    Train loss :  0.42615   val loss :  0.34309
Epoch : 06    Train loss :  0.41156   val loss :  0.34309
Epoch : 07    Train loss :  0.39638   val loss :  0.34309


KeyboardInterrupt: ignored

In [20]:
from copy import deepcopy 
before_tuning_lstm_classifier = deepcopy(lstm_classifier)
before_tuning_lstm_classifier_pool = deepcopy(lstm_pool_classifier)


In [31]:
mode = 'pool'
if mode == 'cola_':
        
    with open (mode +'.dill', 'wb') as f:
        model = {
            'TEXT' : TEXT,
            "LABEL" : LABEL,
            'classifier' : lstm_classifier 
        }
        dill.dump(model, f)

elif mode =='pool':
        
    with open ('pool_model.dill', 'wb') as f:
        model = {
            'TEXT' : TEXT,
            "LABEL" : LABEL,
            'classifier' : lstm_pool_classifier 
        }
        dill.dump(model, f)

In [29]:
%cd /content/

/content


In [22]:
def test(model, test_loader, device):
    model.eval()
    with torch.no_grad():
        y_real = []
        y_pred = []

        for batch in test_loader :
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = model(text).flatten().cpu()

            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)
    
    fpr , tpr , _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)
    return auroc

In [32]:
_ = lstm_pool_classifier.cpu()
test_auroc = test(lstm_pool_classifier, cola_test_iterator, "cpu")

print(f'test AUROC : {test_auroc:.5f}')

test AUROC : 0.54942


In [40]:
def predict_problem(model_path, problem):
    with open(model_path, 'rb') as f:
        model = dill.load(f)
    TEXT = model["TEXT"]
    classifier = model["classifier"]

    problem = list(map(lambda x : x.replace("[","").replace("]",""), problem))
    tokenized_sentances = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentances:
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])

    with torch.no_grad():
        classifier.eval()
        predict = []
        for sentance in sentences:
            sentance = torch.LongTensor([sentance])
            predict += [classifier(sentance).item()]
    
    score_df = pd.DataFrame(predict).T
    score_df.column = [f'answer_{i}thscore' for i in range(1,10)] 

    return score_df

In [42]:
problem = [ 
    "Competitive activities can be more than just performance showcases which the best is recognized and the rest are overlooked.",
    "The provision of timely, constructive feedback to participants on performance is an asset that some competitions and contests offer.",
    "The provision of that type of feedback can be interpreted as shifting the emphasis to demonstrating superior performance but not necessarily excellence.",
    "The emphasis on superiority is what we typically see as fostering a detrimental effect of competition.",
    "Information about performance can be very helpful, not only to the participant who does not win or place but also to those who do.",
    "People from more individualistic cultural contexts tend to be motivated to maintain self-focused agency or control 1 as these serve as the basis of one’s self-worth.",
    "With this form of agency comes the belief that individual successes 2 depending primarily on one’s own abilities and actions, and thus, whether by influencing the environment or trying to accept one’s circumstances, the use of control ultimately centers on the individual.",
    "The independent self may be more 3 driven to cope by appealing to a sense of agency or control.",
    "Research has shown 4 that East Asians prefer to receive, but not seek, more social support rather than seek personal control in certain cases.",
    "Therefore, people 5 who hold a more interdependent self-construal may prefer to cope in a way that promotes harmony in relationships.",
]
problem_label = [0, 1, 1, 1, 1, 1, 0, 1, 1, 1]

answer = predict_problem('/content/pool_model.dill', problem)
display(answer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,0,1,2,3,4,5,6,7,8,9
0,0.918525,0.92542,0.919904,0.928468,0.922836,0.921689,0.921076,0.943399,0.923975,0.922705
